In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%pip install -q -r requirements.txt

In [ ]:
import json
import pandas as pd
import os
import re
import string

from vecsim_app.embeddings import Embeddings
from vecsim_app.data_utils import papers


DATA_PATH = "/home/jovyan/arxiv/arxiv-metadata-oai-snapshot.json"
YEAR_CUTOFF = 2012
YEAR_PATTERN = r"(19|20[0-9]{2})"
ML_CATEGORY = "cs.LG"

In [ ]:
df = pd.DataFrame(papers(data_path=DATA_PATH, year_cutoff=YEAR_CUTOFF, year_pattern=YEAR_PATTERN, ml_category=ML_CATEGORY))
len(df)

In [ ]:
# Avg length of the abstracts
df.abstract.apply(lambda a: len(a.split())).mean()

In [ ]:
df

In [ ]:
df['authors_clean'] = df['authors'].apply(lambda a: ' '.join(re.findall(r'\w\w+', a)).strip())
df['authors_clean'][:3]

In [ ]:
df['text'] = df.apply(lambda r: Embeddings.clean_description(r['authors_clean'] + ' ' + r['title'] + ' ' + r['abstract']), axis=1)
df['text'][:1]

In [ ]:
from sentence_transformers import SentenceTransformer
#see models here: https://www.sbert.net/docs/pretrained_models.html
model_name = 'sentence-transformers/all-distilroberta-v1'

model = SentenceTransformer(model_name)

In [ ]:
# Create embeddings from the title and abstract
emb = model.encode(df['text'].tolist())

In [ ]:
# Add embeddings to df
df = df.reset_index().drop('index', axis=1)
df['vector'] = emb.tolist()

In [ ]:
import pickle

# Export to file!
with open(f'arxiv_embeddings_10000.pkl', 'wb') as f:
    data = pickle.dumps(df)
    f.write(data)

In [ ]:
!ls -lh .